## Storing objects while specifying their vectors

In [16]:
#Loading environment variables

import os

WEAVIATE_CLUSTER_URL = os.getenv("WEAVIATE_CLUSTER_URL")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

## Connect to Weaviate Instance

In [17]:
import weaviate

#Connecting to WCS instance

client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_CLUSTER_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_API_KEY),
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
        "X-Cohere-Api-Key": COHERE_API_KEY
    }
)

client.is_ready()

True

## Create a collection with no Vectorizer

In [18]:
from weaviate.classes.config import Configure, VectorDistances, Property, DataType

if client.collections.exists("MyCollection"):
    client.collections.delete("MyCollection")

client.collections.create(
    name="MyCollection",
    vectorizer_config=Configure.Vectorizer.none(),
    vector_index_config=Configure.VectorIndex.hnsw(
        distance_metric=VectorDistances.COSINE
    ),
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="foo", data_type=DataType.INT),
    ]
)

print("Collection creation success")

Collection creation success


In [19]:
my_collection = client.collections.get("MyCollection")

## Insert object with Vector

In [20]:
my_collection.data.insert(
    properties={
        "title": "First Object",
        "foo": 11
    },
    vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
)

UUID('ae11fb7b-4bde-4466-b7a6-bf6cabab2757')

## Fetch objects with vectors

In [21]:
response = my_collection.query.fetch_objects(limit=1, include_vector=True)
print(response.objects[0].vector)

{'default': [0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612]}


## Insert many objects with their vectors using batch

In [22]:
source = [
    {
        "title": "Second Object",
        "foo": 22,
        "vector": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    },
    {
        "title": "Third Object",
        "foo": 33,
        "vector": [0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
    },
    {
        "title": "Fourth Object",
        "foo": 44,
        "vector": [0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
    },
    {
        "title": "Fifth Object",
        "foo": 55,
        "vector": [0.5, 0.5, 0, 0, 0, 0]
    },
]

In [23]:
with my_collection.batch.dynamic() as batch:
    # TODO:  iterate through objects here and add them using batch
    for item in source:
        batch.add_object(
            properties={
                "title": item["title"],
                "foo": item["foo"]
            },
            vector=item["vector"]
        )

In [24]:
response = my_collection.query.fetch_objects(limit=5, include_vector=True)
print(response.objects[2].vector)

{'default': [0.4000000059604645, 0.4099999964237213, 0.41999998688697815, 0.4300000071525574, 0.4399999976158142, 0.44999998807907104]}


## Using insert_many

In [25]:
# sample_data = [
#    wc.DataObject(
#       properties={
#          "title": "First Object",
#          "foo": 11, 
#       },
#       vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Second Object",
#          "foo": 22,
#       },
#       vector=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Third Object",
#          "foo": 33,
#       },
#       vector=[0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Fourth Object",
#         "foo": 44,
#       },
#       vector=[0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Fifth Object",
#          "foo": 55,
#       },
#       vector=[0.5, 0.5, 0, 0, 0, 0]
#    ),
# ]

# my_collection.data.insert_many(sample_data)

## Query
Available types of queries you can run when working with vector embeddings (without modules) in **Weaviate**:

1. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#search-with-a-vector)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#search-with-an-existing-object)

## near_vector

In [26]:
response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5]
)

for item in response.objects:
    print(item.uuid)
    print(item.properties, "\n")

c75ab1e0-3742-499f-9456-72e7aead5c00
{'title': 'Second Object', 'foo': 22} 

bd3b882b-10d3-4722-b4ff-6eafadea6240
{'title': 'Fourth Object', 'foo': 44} 

ae11fb7b-4bde-4466-b7a6-bf6cabab2757
{'title': 'First Object', 'foo': 11} 

9b1452f9-d87d-48d8-a47e-39f2f8dec7f4
{'title': 'Fifth Object', 'foo': 55} 

dcfb306d-6922-47ae-a91d-808fc9a5671e
{'title': 'Third Object', 'foo': 33} 



In [27]:
from weaviate.classes.query import MetadataQuery, Filter


response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    include_vector=True, 
    limit=3,
    filters=Filter.by_property("foo").greater_than(11),
    return_metadata=MetadataQuery(distance=True),# TODO: request distance from metadata
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance)
    print(item.vector, "\n")

{'title': 'Second Object', 'foo': 22}
0.6506307125091553
{'default': [0.20000000298023224, 0.30000001192092896, 0.4000000059604645, 0.5, 0.6000000238418579, 0.699999988079071]} 

{'title': 'Fourth Object', 'foo': 44}
0.8072028756141663
{'default': [0.4000000059604645, 0.4099999964237213, 0.41999998688697815, 0.4300000071525574, 0.4399999976158142, 0.44999998807907104]} 

{'title': 'Fifth Object', 'foo': 55}
0.9925509691238403
{'default': [0.5, 0.5, 0.0, 0.0, 0.0, 0.0]} 



In [28]:
from weaviate.classes.query import Filter, MetadataQuery

response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    return_metadata=MetadataQuery(distance=True),
    limit=2,
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance, "\n")

{'title': 'Second Object', 'foo': 22}
0.6506307125091553 

{'title': 'Fourth Object', 'foo': 44}
0.8072028756141663 



## near_object

In [30]:
from weaviate.classes.query import MetadataQuery

response = my_collection.query.near_object(
    near_object="9b1452f9-d87d-48d8-a47e-39f2f8dec7f4", #uuid for fifth object
    return_metadata=MetadataQuery(distance=True),
    limit=3
)

for item in response.objects:
    print(item.uuid)
    print(item.properties)
    print(item.metadata.distance, "\n")

9b1452f9-d87d-48d8-a47e-39f2f8dec7f4
{'title': 'Fifth Object', 'foo': 55}
5.960464477539063e-08 

ae11fb7b-4bde-4466-b7a6-bf6cabab2757
{'title': 'First Object', 'foo': 11}
0.4226497411727905 

bd3b882b-10d3-4722-b4ff-6eafadea6240
{'title': 'Fourth Object', 'foo': 44}
0.4502628445625305 

